# Régression simple

## Objectif:
- **Développer un modèle de Machine Learning, permettant d'estimer le poids d'un crâne en fonction de son volume**

## Importer les librairies pour lire, explorer et manipuler les données

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ignorer les méssages d'avertissements
import warnings
warnings.filterwarnings('ignore')

from IPython.display import Image

%matplotlib inline

## Importation et exploration  des données

In [ ]:
data = pd.read_csv('headbrain.csv')

In [ ]:
print(data.shape) # Format de la source (nombre de lignes, nombre de colonnes)
data.info()       # Affiche des  informations sur les données
data.head() 

#### Sélection des champs utiles

In [ ]:
data = data.drop(['Gender', 'Age Range'], axis=1)
data.head()

#### Détection des données dupliquées, manquantes

In [ ]:
#Comptage du nombre de ligne duppiquées
print ("Nombre de données dupliquées:", data.duplicated().sum())
# Détection des valeurs manquantes
print ("Nombre de données manquantes:\n", data.isnull().sum())

#### Détection des données aberrantes

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5),gridspec_kw={'wspace': 0.3})
sns.boxplot(data['Head Size(cm^3)'], ax = axs[0])
sns.boxplot(data['Brain Weight(grams)'], ax = axs[1])
plt.show()

#### Analyse de la relation entre les deux variables

In [ ]:
sns.scatterplot(data['Head Size(cm^3)'], data['Brain Weight(grams)']);

In [ ]:
corr = data.corr()
corr

## Préparation des données

#### Détection des valeurs aberante dans X=Head Size(cm^3) par la methode IQR

In [ ]:
#Premier et Troisième quartile de la variable Head Size(cm^3)
Q1 = data['Head Size(cm^3)'].quantile(0.25)
Q3 = data['Head Size(cm^3)'].quantile(0.75)
IQR = Q3 - Q1
minimum = Q1 - 1.5*IQR
maximum= Q3 + 1.5*IQR

- Les valeurs aberrantes de la variable: Head Size(cm^3)

In [ ]:
outlier_Head = data[(data['Head Size(cm^3)']<minimum)|(data['Head Size(cm^3)']>maximum)]
outlier_Head.shape, outlier_Head.index

In [ ]:
outlier_Head

#### Détection des valeurs aberante dans y= Brain Weight(grams) par la methode IQR

In [ ]:
Q1 = data['Brain Weight(grams)'].quantile(0.25)
Q3 = data['Brain Weight(grams)'].quantile(0.75)
IQR = Q3 - Q1
minimum = Q1 - 1.5*IQR
maximum = Q3 + 1.5*IQR

- Les valeurs aberantes dans la variable: Brain Weight(grams)

In [ ]:
outliers_Brain= data[(data['Brain Weight(grams)']<minimum)|(data['Brain Weight(grams)']>maximum)]
outliers_Brain.shape, outliers_Brain.index

In [ ]:
outliers_Brain

- DataFrame des valeurs aberrantes

In [ ]:
outliers = pd.concat([outlier_Head, outliers_Brain])
outliers.shape, outliers.index

In [ ]:
#Suppréssion des doublons
outliers= outliers.drop_duplicates()
outliers.shape

#### Suppression des données aberrantes

In [ ]:
data_nettoye = data.drop(outliers.index)
data_nettoye.shape

## Modélisation

#### Importer les librairies pour la modélisation les testes 

In [ ]:
# Préparer une base de test et une base d'apprentissage
from sklearn import model_selection
from sklearn.model_selection import train_test_split 

# Importer la classe LinearRegression
from sklearn.linear_model import LinearRegression 

#Importer les metriques pour tester le modèle
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

#### Diviser la base en un ensemble d'apprentissage et un ensemble de test (par defaut 75 % pour l'apprentissage et 25% pour le test)

In [ ]:
# Variable d'entrée (explicative)
X= data[['Head Size(cm^3)']] 
# Variable de sortie à expliquer
y= data[['Brain Weight(grams)']] 

# Création d'une base d'apprentissage et d'une base de test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.30, random_state = 42)
X_train.shape , y_train.shape, X_test.shape, y_test.shape

#### Recherche du modèle y = a*X +b

In [ ]:
# Création du modèle
rl = LinearRegression()
# Apprentissage du modèle utilisant la base d'apprentissage
rl.fit(X_train,y_train)
# Estimation de y utilisant la base de test 
y_pred = rl.predict(X_test)

#Calcul des paramètres du modèle
a = rl.coef_   
b = rl.intercept_
print ("a=", a)
print("b=", b)


#### Qualité du modèle developé

In [ ]:
Image('Metrics.PNG')

In [ ]:
# Clacul de MAE, MSE, RMSE, R et R²

mae = mean_absolute_error(y_test, y_pred) # Calcul l'erreur absolue moyenne
mse = mean_squared_error(y_test, y_pred) # Calcul l’erreur quadratique moyenne
rmse = np.sqrt(mse) #Racine carrée de l’erreur quadratique moyenne
r2_score = rl.score(X_test, y_test) # Calcule de R²
r_score = np.sqrt(r2_score) # Calcul de R
print ("MAE=", mae)
print ("MSE=", mse)
print("RMSE=" , np.sqrt(mse))
print("R= ", r_score)
print("R²= ", r2_score)

#### Modélisation  sans les valeurs aberantes

In [ ]:
X= data_nettoye[['Head Size(cm^3)']] # Variable d'entrée (explicative)
y= data_nettoye[['Brain Weight(grams)']] # Variable de sortie à expliquer
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.30, random_state = 42)

In [ ]:
# Création du modèle
rl = LinearRegression()
# Apprentissage du modèle utilisant la base d'apprentissage
rl.fit(X_train,y_train)
# Estimation de y utilisant le modèle developpé
y_pred = rl.predict(X_test)

#Calcul des paramètres du modèle
a = rl.coef_   
b = rl.intercept_
print ("a=", a)
print("b=", b)

In [ ]:
# Clacul de MAE, MSE, RMSE, R et R²

mae = mean_absolute_error(y_test, y_pred) # Calcul l'erreur absolue moyenne
mse = mean_squared_error(y_test, y_pred) # Calcul l’erreur quadratique moyenne
rmse = np.sqrt(mse) #Racine carrée de l’erreur quadratique moyenne
r2_score = rl.score(X_test, y_test) # Calcule de R²
r_score = np.sqrt(r2_score) # Calcul de R
print ("MAE=", mae)
print ("MSE=", mse)
print("RMSE=" , np.sqrt(mse))
print("R= ", r_score)
print("R²= ", r2_score)

## Régression lineaire multiple

## Objectif:
- **Développer un modèle de Machine Learning, permettant d'estimer les ventes en fonction de plusieurs variables**

## Importer et explorer  la source de données

In [ ]:
df = pd.read_csv('Advertising.csv')
print(df.shape) # Format de la source (nombre de lignes, nombre de colonnes)
df.info()       # Affiche des  informations sur les données
df.head()       # Affiche les 5 premiers enregistrements

#### Détection des valeurs manquantes, aberrantes, des doublons 

In [ ]:
#Comptage du nombre de ligne duppiquées
df.duplicated().sum()

In [ ]:
# Détection des valeurs manquantes
df.isnull().sum()

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(10,8),gridspec_kw={'hspace': 0.2, 'wspace': 0.2})
sns.boxplot(df['TV'], ax = axs[0][0])
sns.boxplot(df['radio'], ax = axs[0][1])
sns.boxplot(df['newspaper'], ax = axs[1][0])
sns.boxplot(df['sales'], ax = axs[1][1])
plt.show()

#### Analyse des correlations entre les variables

In [ ]:
sns.pairplot(df, diag_kind = 'kde')
plt.show()

In [ ]:
#Matrice de correlation
Mat_Corr = df.corr()
Mat_Corr

In [ ]:
sns.heatmap(Mat_Corr, 
            xticklabels=Mat_Corr.columns,
            yticklabels=Mat_Corr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.7);

### Préparation des données

#### Diviser la base en un ensemble d'apprentissage et un ensemble de test 

In [ ]:
X= df[['TV', 'radio', 'newspaper']] # Les variables d'entrées
Xr=df[['TV', 'radio']] # Les variables d'entrées sans la variable newspaper
y= df[['sales']] # La variable de sortie

#Les ensembles d'apprentisssage et de test avec tous les variables
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.30, random_state = 42)
#Les ensembles d'apprentisssage et de test sans la variable newspaper
Xr_train, Xr_test, yr_train, yr_test = model_selection.train_test_split(Xr, y, test_size = 0.30, random_state = 41)

In [ ]:
X_train.shape , y_train.shape, X_test.shape, y_test.shape

In [ ]:
Xr_train.shape , yr_train.shape, Xr_test.shape, yr_test.shape

#### Recherche des modèles;
- Sales = a0+ a1 * TV + a2 * radio + a3 * newspaper
- Sales_r = a0+ a1 * TV + a2 * radio 

In [ ]:
# Création des modèles
lr1 = LinearRegression()
lr2 = LinearRegression()

# Apprentissage du modèle utilisant la base d'apprentissage
lr1.fit(X_train,y_train)
# Apprentissage du modèle utilisant la base d'apprentissage (sans la variable newspaper)
lr2.fit(Xr_train,yr_train)

# Estimation de y modèle entier
y_pred = lr1.predict(X_test)
# Estimation de y modèle réduit
yr_pred = lr2.predict(Xr_test)

In [ ]:
#Calcul des paramètres du modèle entier
print('Les paramètre du modèle entier:')
print('-------------------------------------\n')
a0 = lr1.intercept_   
a1 = lr1.coef_[0][0]
a2 = lr1.coef_[0][1]
a3 = lr1.coef_[0][2]
print("a0=", a0)
print("a1=", a1)
print("a2=", a2)
print("a3=", a3)
print('\nLes paramètre du modèle sans la variable newspapers:')
print('---------------------------------------------------------')
#Calcul des paramètres du modèle réduit
a0_r = lr2.intercept_   
a1_r = lr2.coef_[0][0]
a2_r = lr2.coef_[0][1]
print("a0_r=", a0_r)
print("a1_r=", a1_r)
print("a2_r=", a2_r)

#### Qualité des modèles developés

In [ ]:
# Clacul de MAE, MSE, RMSE, R et R² pour le modèle entier

print("Qualité du modèle entier:")
print('-------------------------------------')
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2_score = lr1.score(X_test, y_test)
r_score = np.sqrt(r2_score)
print ("MAE=", mae)
print ("MSE=", mse)
print("RMSE=" , np.sqrt(mse))
print("R= ", r_score)
print("R²= ", r2_score)

# Clacul de MAE, MSE, RMSE, R et R² pour le modèle réduit
print("\nQualité du modèle réduit:")
print('-------------------------------------')
mae = mean_absolute_error(yr_test, yr_pred)
mse = mean_squared_error(yr_test, yr_pred)
rmse = np.sqrt(mse)
r2_score = lr2.score(Xr_test, yr_test)
r_score = np.sqrt(r2_score)
print ("MAE_r=", mae)
print ("MSE_r=", mse)
print("RMSE_r=" , np.sqrt(mse))
print("R= ", r_score)
print("R²= ", r2_score)

## Autres modèles de régression

### Modèle de régression utilisant  Les arbres de décision

In [ ]:
#Importation de la librairie
from sklearn.tree import DecisionTreeRegressor

In [ ]:
X= df[['TV', 'radio', 'newspaper']] # Les variables d'entrées
y= df[['sales']] # La variable de sortie

#Les ensembles d'apprentisssage et de test avec tous les variables
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.30, random_state = 42)

In [ ]:
#Modélisation
tree = DecisionTreeRegressor()
tree.fit(X_train,y_train)
pred_tree= tree.predict(X_test)

# Clacul de MAE, MSE, RMSE, R et R² 

mae_TREE = mean_absolute_error(y_test, pred_tree)
mse_TREE = mean_squared_error(y_test, pred_tree)
rmse_TREE= np.sqrt(mse_TREE)
r2_score = tree.score(X_test, y_test)
r_score = np.sqrt(r2_score)
print ("MAE Modèle TREE=", mae_TREE)
print ("MSE Modèle TREE=", mse_TREE)
print("RMSE Modèle TREE=" , rmse_TREE)
print("R= ", r_score)
print("R²= ", r2_score)

### Modèle de régression utilisant  Les forêts aléatoires

In [ ]:
Image('RF_Regressor.PNG')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()# 100 arbres, valeur par défaut
rf.fit(X_train,y_train)
pred_rf= rf.predict(X_test)

In [ ]:
# Clacul de MAE, MSE, RMSE, R et R² 

mae_RF_100 = mean_absolute_error(y_test, pred_rf)
mse_RF_100= mean_squared_error(y_test, pred_rf)
rmse_RF_100 = np.sqrt(mse_RF_100)
r2_score = rf.score(X_test, y_test)
r_score = np.sqrt(r2_score)
print ("MAE RF(100 arbres) =", mae_RF_100)
print ("MSE RF(100 arbres) =", mse_RF_100)
print("RMSE RF (100 arbres) =" ,rmse_RF_100)
print("R= ", r_score)
print("R²= ", r2_score)